### Restart and Run All

In [1]:
import pandas as pd
from datetime import date, timedelta
from sqlalchemy import create_engine

engine = create_engine('sqlite:///c:\\ruby\\portlt\\db\\development.sqlite3')
conlt = engine.connect()

pd.set_option('display.float_format','{:,.2f}'.format)
pd.set_option('display.max_rows',None)

year = 2022
quarter = 2

today = date.today()
today

datetime.date(2022, 8, 12)

In [2]:
sql = '''
SELECT name,year,quarter,q_amt 
FROM epss 
WHERE (year = %s AND quarter <= %s)
OR (year = %s-1 AND quarter >= %s+1) 
ORDER BY year DESC, quarter DESC'''
sql = sql % (year, quarter, year, quarter)
dfc = pd.read_sql(sql, conlt)
dfc['Counter'] = 1
dfc_grp = dfc.groupby(['name'], as_index=False).sum()
dfc_grp = dfc_grp[dfc_grp['Counter'] == 4]
dfc_grp

,name,year,quarter,q_amt,Counter
0,ACE,8086,10,1485354,4
1,ADVANC,8086,10,25853429,4
4,AIE,8086,10,402015,4
5,AIMIRT,8086,10,566114,4
6,AIT,8086,10,556639,4
7,AJ,8086,10,422747,4
8,AMATA,8086,10,2171767,4
9,ANAN,8086,10,-796635,4
10,AOT,8086,10,-16784073,4
11,AP,8086,10,5329148,4


In [3]:
sql = '''
SELECT name,year,quarter,q_amt 
FROM epss 
WHERE (year = %s AND quarter <= %s-1) 
OR (year = %s-1 AND quarter >= %s) 
ORDER BY year DESC, quarter DESC'''
sql = sql % (year, quarter, year, quarter)
print(sql)


SELECT name,year,quarter,q_amt 
FROM epss 
WHERE (year = 2022 AND quarter <= 2-1) 
OR (year = 2022-1 AND quarter >= 2) 
ORDER BY year DESC, quarter DESC


In [4]:
dfp = pd.read_sql(sql, conlt)
dfp['Counter'] = 1
dfp_grp = dfp.groupby(['name'], as_index=False).sum()
dfp_grp = dfp_grp[dfp_grp['Counter'] == 4]
dfp_grp.shape

(220, 5)

In [5]:
dfm = pd.merge(dfc_grp, dfp_grp, on='name', suffixes=(['_c','_p']), how='inner')
dfm['inc_profit'] = dfm['q_amt_c']-dfm['q_amt_p']
dfm['Pct'] = round(dfm['inc_profit']/abs(dfm['q_amt_p'])*100,2)
dfm['year'] = year
dfm['quarter'] = 'Q'+str(quarter)
df_pct = dfm[['name','year','quarter','q_amt_c','q_amt_p','inc_profit','Pct']]
df_pct

,name,year,quarter,q_amt_c,q_amt_p,inc_profit,Pct
0,ACE,2022,Q2,1485354,1468062,17292,1.18
1,ADVANC,2022,Q2,25853429,26589096,-735667,-2.77
2,AIE,2022,Q2,402015,432491,-30476,-7.05
3,AIMIRT,2022,Q2,566114,524978,41136,7.84
4,AIT,2022,Q2,556639,571977,-15338,-2.68
5,AJ,2022,Q2,422747,467710,-44963,-9.61
6,AMATA,2022,Q2,2171767,1770024,401743,22.70
7,ANAN,2022,Q2,-796635,-744140,-52495,-7.05
8,AOT,2022,Q2,-16784073,-17151697,367624,2.14
9,AP,2022,Q2,5329148,4870212,458936,9.42


In [6]:
sql = """
DELETE FROM qt_profits 
WHERE year = %s AND quarter = 'Q%s'"""
sql = sql % (year,quarter)
print(sql)


DELETE FROM qt_profits 
WHERE year = 2022 AND quarter = 'Q2'


In [7]:
rp = conlt.execute(sql)
rp.rowcount

113

In [8]:
sql = 'SELECT name, id FROM tickers'
tickers = pd.read_sql(sql, conlt)
tickers.shape

(402, 2)

In [9]:
df_ins = pd.merge(df_pct, tickers, on='name', how='inner')
df_ins.dtypes

name           object
year            int64
quarter        object
q_amt_c         int64
q_amt_p         int64
inc_profit      int64
Pct           float64
id              int64
dtype: object

In [10]:
rcds = df_ins.values.tolist()
len(rcds)

174

In [11]:
for rcd in rcds:
    print(rcd)

['ACE', 2022, 'Q2', 1485354, 1468062, 17292, 1.18, 698]
['ADVANC', 2022, 'Q2', 25853429, 26589096, -735667, -2.77, 6]
['AIE', 2022, 'Q2', 402015, 432491, -30476, -7.05, 720]
['AIMIRT', 2022, 'Q2', 566114, 524978, 41136, 7.84, 669]
['AIT', 2022, 'Q2', 556639, 571977, -15338, -2.68, 11]
['AJ', 2022, 'Q2', 422747, 467710, -44963, -9.61, 12]
['AMATA', 2022, 'Q2', 2171767, 1770024, 401743, 22.7, 21]
['ANAN', 2022, 'Q2', -796635, -744140, -52495, -7.05, 23]
['AOT', 2022, 'Q2', -16784073, -17151697, 367624, 2.14, 24]
['AP', 2022, 'Q2', 5329148, 4870212, 458936, 9.42, 25]
['ASIAN', 2022, 'Q2', 1048954, 1077825, -28871, -2.68, 36]
['ASK', 2022, 'Q2', 1367575, 1277644, 89931, 7.04, 38]
['ASP', 2022, 'Q2', 578771, 775000, -196229, -25.32, 40]
['ASW', 2022, 'Q2', 871576, 856966, 14610, 1.7, 728]
['AWC', 2022, 'Q2', 3075541, 2100590, 974951, 46.41, 699]
['BA', 2022, 'Q2', -8985803, -8824265, -161538, -1.83, 45]
['BAM', 2022, 'Q2', 2705410, 2665001, 40409, 1.52, 709]
['BANPU', 2022, 'Q2', 30044483, 

In [12]:
sql = """
INSERT INTO qt_profits (name, year, quarter, latest_amt, previous_amt, inc_amt, inc_pct, ticker_id) \
 VALUES (?, ?, ?, ?, ?, ?, ?, ?)"""
print(sql)


INSERT INTO qt_profits (name, year, quarter, latest_amt, previous_amt, inc_amt, inc_pct, ticker_id)  VALUES (?, ?, ?, ?, ?, ?, ?, ?)


In [13]:
for rcd in rcds:
    conlt.execute(sql, rcd)

### End of loop

In [14]:
cols = 'name year quarter q_amt_c q_amt_p inc_profit Pct'.split()

In [15]:
criteria_1 = df_ins.q_amt_c > 440000
df_ins.loc[criteria_1,cols].sort_values(['Pct'],ascending=[False]).head()

,name,year,quarter,q_amt_c,q_amt_p,inc_profit,Pct
89,M,2022,Q2,851758,313003,538755,172.12
158,TOP,2022,Q2,39604949,16400690,23204259,141.48
148,TFG,2022,Q2,1543647,734740,808907,110.09
52,ESSO,2022,Q2,14995042,7554525,7440517,98.49
39,CPNREIT,2022,Q2,1624073,869788,754285,86.72


In [16]:
criteria_2 = df_ins.q_amt_p > 400000
df_ins.loc[criteria_2,cols].sort_values(['Pct'],ascending=[False]).head()

,name,year,quarter,q_amt_c,q_amt_p,inc_profit,Pct
158,TOP,2022,Q2,39604949,16400690,23204259,141.48
148,TFG,2022,Q2,1543647,734740,808907,110.09
52,ESSO,2022,Q2,14995042,7554525,7440517,98.49
39,CPNREIT,2022,Q2,1624073,869788,754285,86.72
135,SPRC,2022,Q2,14407821,8025156,6382665,79.53


In [17]:
criteria_3 = df_ins.Pct > 10
df_ins.loc[criteria_3,cols].sort_values(['Pct'],ascending=[False]).head()

,name,year,quarter,q_amt_c,q_amt_p,inc_profit,Pct
89,M,2022,Q2,851758,313003,538755,172.12
158,TOP,2022,Q2,39604949,16400690,23204259,141.48
148,TFG,2022,Q2,1543647,734740,808907,110.09
52,ESSO,2022,Q2,14995042,7554525,7440517,98.49
39,CPNREIT,2022,Q2,1624073,869788,754285,86.72


In [18]:
df_ins_criteria = criteria_1 & criteria_2 & criteria_3
df_ins.loc[df_ins_criteria,cols].sort_values(['Pct'],ascending=[False]).head()

,name,year,quarter,q_amt_c,q_amt_p,inc_profit,Pct
158,TOP,2022,Q2,39604949,16400690,23204259,141.48
148,TFG,2022,Q2,1543647,734740,808907,110.09
52,ESSO,2022,Q2,14995042,7554525,7440517,98.49
39,CPNREIT,2022,Q2,1624073,869788,754285,86.72
135,SPRC,2022,Q2,14407821,8025156,6382665,79.53
